In [1]:
import re
import math
import fitz
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.matcher import Matcher
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
import es_core_news_sm
from nltk.tokenize import sent_tokenize
import os
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import es_core_news_sm
import itertools
from nltk.stem import SnowballStemmer
import textacy
import regex
import json
import unidecode
import numpy as np
from gensim.models.keyedvectors import KeyedVectors

import yaml
wordvectors_file_vec ='/home/erwin/Genoma/cv-parser/parser/embeddings/fasttext-sbwc.3.6.e20.vec'

In [2]:
cantidad = 300000

model = KeyedVectors.load_word2vec_format(wordvectors_file_vec, limit=cantidad)

In [2]:
path_to_json = '/home/erwin/Genoma/cv-parser/parser/Outputs/output_seccionado'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]
#print(json_files)  # for me this prints ['foo.json']
jsons = []
for index, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        jsons.append(json.load(json_file))
        


In [3]:
jsons[0]['experiencia']

'marketing gestión comercial persona proactiva ordenar alto sentir responsabilidad equipar fundamental empresa orientar resultar tangible laboral adecco peru cargo gerente desarrollo fecha 01/04/2019 fecho responsable manejar cartero top cliente responsable cumplimiento objetivo venta líneo negociar garantizar rentabilidad nivel servicio desarrollo convenio diverso institución implementación canal post-venta cliente extranjero enfoque negociar cliente extranjero desarrollar servicio medir cliente adecco peru cargo gerente sucursal fecha 01/01/2017 31/03/2019 responsable gestión integral sucursal responsable cumplimiento objetivo venta líneo negociar garantizar rentabilidad nivel servicio desarrollo ejecución plan negocio venta kpis estrategia comercial orientar captación mantenimiento retención cliente cargo ejecutivo comercial fecha 25/03/2015 31/12/2016 funciones ejecutar estrategia comercial orientar captación mantenimiento retención cliente sucursal identificar información negocio 

In [4]:
sent = pre_process(jsons[0]['experiencia'])
sent

NameError: name 'pre_process' is not defined

In [5]:
sent_lem = lematizar(sent)
print(sent_lem)

NameError: name 'lematizar' is not defined

In [6]:
lematizar('ingles')

NameError: name 'lematizar' is not defined

In [7]:
def sent2vec(s):
    '''Generate Vectora for sentences.'''
    M = []
    for w in s.split():
        try:
            M.append(model[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    return v/np.sqrt((v**2).sum())

def cosine_sim(vec1, vec2):
    '''Return Cosine Similarity.'''
    return  np.dot(vec1,vec2)/(np.linalg.norm(vec1)* np.linalg.norm(vec2))

def get_closest(word, n):
    '''Get n most similar words by words.'''
    #This function can easily be expanded to get similar words to phrases--
    #using sent2vec() method defined in WithWord2Vec notebook. 
    word = word.lower()
    words = [word]
    similar_vals = [1]
    try:
        similar_list = model.most_similar(positive=[word],topn=n)
        
        for tupl in similar_list:
            words.append(tupl[0])
            similar_vals.append(tupl[1])
    except:
        #If word not in vocabulary return same word and 1 similarity-- 
        #see initialisation of words, similarities.
        pass
    
    return words, similar_vals

In [8]:
frase_1 = 'software developer'
frase_2 = 'web developer'


vector_sentence_1 = sent2vec(frase_1)
vector_sentence_2 = sent2vec(frase_2)
similitud = cosine_sim(vector_sentence_1,vector_sentence_2)

print(similitud)

nan


<ipython-input-7-7bd0ad79f7ac>:11: RuntimeWarning: invalid value encountered in double_scalars
  return v/np.sqrt((v**2).sum())


In [9]:
get_closest(word= 'ingle', n = 3)

(['ingle'], [1])

In [10]:
model.similarity('tradición', 'tradicional')

NameError: name 'model' is not defined

In [11]:
import es_core_news_md
nlp = es_core_news_md.load()
nlp_text = nlp('educado')
nlp_text[0].lemma_

'educar'

In [12]:
prc_description = '''ingeniería máster postgrado excel desarrollo gestión comercial experiencia manejo clientes
emprendimiento liderar equipos planificar organizar dirigir trabajo presión seguimiento KPI inglés '''

In [13]:
# https://github.com/prateekguptaiiitk/Resume_Filtering/blob/develop/Scoring/CV_ranking.ipynb
word_value = {}
similar_words_needed = 1
for word in prc_description.split():
    similar_words, similarity = get_closest(word, similar_words_needed)
    for i in range(len(similar_words)):
        word_value[similar_words[i]] = word_value.get(similar_words[i], 0)+similarity[i]
        #print(similar_words[i], word_value[similar_words[i]])
        #print('------------------------------------------------')

In [14]:
word_value.keys()

dict_keys(['ingeniería', 'máster', 'postgrado', 'excel', 'desarrollo', 'gestión', 'comercial', 'experiencia', 'manejo', 'clientes', 'emprendimiento', 'liderar', 'equipos', 'planificar', 'organizar', 'dirigir', 'trabajo', 'presión', 'seguimiento', 'kpi', 'inglés'])

In [15]:
### ahora veamos si resulta el ranking
#frecuencia de término – frecuencia inversa de documento 
#Tf-idf
#Para calcular este ranking es mejor tener las secciones skills y experiencia
#con el fin de calcular esta metrica usando la ocurrencia de las palabras
# Tenemos todos los CV's y una descripción del cargo, a este descripcion del cargo
# tiene N palabras, le buscamos 2 palabras parecidas, generando una descripcion
# de N*2

# Usando esta nuevo set de palabras de descripcion, recorremos todos los cvs contando 
# la ocurrencia de estas palabras en cada documento, y luego se genera un ranking usando Tf-idf
# La pregunta es: ¿Lo haré sobre el documento entero? o ¿Trataré de seccionar y ocupar ciertas secciones?

no_of_cv = len(jsons)
#print(no_of_cv)

count = {}
idf = {}
for word in word_value.keys():
    count[word] = 0
    for i in range(no_of_cv):
        try:
            if word in jsons[i]['skills'] or word in jsons[i]['experiencia'] or word in jsons[i]['educación']:
                #print('entre')
                count[word] += 1
        except:
            pass

    idf[word] = math.log((no_of_cv+1)/(1+count[word]))
    #print(idf)

In [16]:
#calculemos usando similitud



def calculo_similitud(word1, array_palabras):
    n_veces = 0
    for word in array_palabras:
        try:
            sim = model.similarity(word, word1)
            #print(word)
            #print(sim)
            if sim > 0.5:
                n_veces += 1
            else:
                continue
        except: #No estaba la palabra
            pass
        
    return n_veces



word1 = 'feliz'
array = ['alegre', 'contento', 'taladro', 'juguete']




n = calculo_similitud(word1, array)
n

0

In [17]:
jsons[0]['skills'].split()

[]

In [18]:
no_of_cv = len(jsons)

count = {}
idf = {}
for word in word_value.keys():
    count[word] = 0
    for i in range(no_of_cv):
        # eliminación de stopwords y quizas lematizacion
        skill_pro = pre_process(jsons[i]['skills']) 
        expe_pro = pre_process(jsons[i]['experiencia'])
        edu_pro = pre_process(jsons[i]['educación'])
        
        if calculo_similitud(word, skill_pro.split()) or calculo_similitud(word, expe_pro.split()) or calculo_similitud(word, edu_pro.split()):
            count[word] += 1


    idf[word] = math.log((no_of_cv+1)/(1+count[word]))

NameError: name 'pre_process' is not defined

In [ ]:
idf

In [ ]:

score = {}
for i in range(no_of_cv):
    score[i] = 0
    skill_pro = pre_process(jsons[i]['skills']) 
    expe_pro = pre_process(jsons[i]['experiencia'])
    edu_pro = pre_process(jsons[i]['educación'])
    for word in word_value.keys():
        
        
        
        tf = 1 + calculo_similitud(word, skill_pro.split()) + calculo_similitud(word, expe_pro.split()) + calculo_similitud(word, edu_pro.split())

        score[i] += word_value[word]*tf*idf[word]

print(score)

In [ ]:
sorted_list = []
for i in range(no_of_cv):
    sorted_list.append((score[i], jsons[i]['nombre archivo']))
    
sorted_list.sort(reverse = True)


In [ ]:
sorted_list

In [ ]:
sorted_list

In [ ]:
#sorted_list

In [ ]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from unidecode import unidecode
import string
import spacy
import es_core_news_sm


def pre_process(corpus):
    corpus = corpus.lower()

    stopset = stopwords.words('spanish') + list(string.punctuation)

    corpus = " ".join([i for i in word_tokenize(corpus) if i not in stopset])
    # remove non-ascii characters
    corpus = unidecode(corpus)
    return corpus

def lematizar(frase):
    nlp = es_core_news_sm.load()
    doc = nlp(frase)
    lemmas = [tok.lemma_.lower() for tok in doc]
    return lemmas



In [ ]:

test = pre_process('educación básica')
test


In [19]:
test = lematizar('ingeniero')
test

NameError: name 'lematizar' is not defined

In [20]:
lematizar('ingeniero') in lematizar('ingenieria')

NameError: name 'lematizar' is not defined

In [21]:
model.similarity('ingeniero', 'ingenieria')

NameError: name 'model' is not defined

In [22]:
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('spanish')
   # stemmed_claves = [stemmer.stem(token) for token in palabras_claves]
#stop_words = set(stopwords.words('spanish')) 

[stemmer.stem(x) for x in test] 
#stemmed_clave

NameError: name 'test' is not defined

In [23]:
stemmer.stem('ingeniería') in stemmer.stem('ingeniería')

True

In [24]:
import os
import nltk
import string
from nltk.tokenize import sent_tokenize, word_tokenize

def cargar_dict(path):
    with open(path) as f:  
        array = [x.strip() for x in f]
        c = [x for x in array if x != ''] # '' aparece cuando hay lines vacias
    return c

In [25]:
newStopWords = cargar_dict('/home/erwin/Genoma/cv-parser/parser/diccionarios/stop_words')
stopwords = nltk.corpus.stopwords.words('spanish')
stopwords.extend(newStopWords)

FileNotFoundError: [Errno 2] No such file or directory: '/home/erwin/Genoma/cv-parser/parser/diccionarios/stop_words'

In [26]:



def pre_process(corpus, stopWords, enminiscula = True):
    if enminiscula:
        corpus = corpus.lower()
    stopset = stopwords+ list(string.punctuation)

    corpus = " ".join([i for i in word_tokenize(corpus) if i not in stopset])
    # remove non-ascii characters
    #corpus = unidecode.unidecode(corpus)
    return corpus

In [27]:
pre_process('UNIVERSIDAD Universidad universidad', stopwords, False)

TypeError: unsupported operand type(s) for +: 'LazyCorpusLoader' and 'list'

In [28]:
import es_core_news_sm
def lematizar(frase):
    '''
    Esta función recibe un string y le aplica lematización:
    ingeniero ---> ingenier
    ingeniera ----> ingenier
    '''
    nlp = es_core_news_sm.load()
    doc = nlp(frase)
    lemmas = [tok.lemma_.lower() for tok in doc]
    return lemmas

In [39]:
des = '''
Nos encontramos en la búsqueda de Socios comerciales, para administrar por completo importantes puntos de ventas (corner) de retail de calzado.

Has escuchado hablar de las franquicias y consignaciones? pues esta empresa trabaja a través de la consignación, esto quiere decir que la empresa te hace entrega del punto de venta, corriendo con todos los gastos de infraestructura, arriendo y toda la mercadería y renovación de stock constante, apoyo de visual y marketing,etc.
Buscamos vendedores que se encarguen de la administración por completo del punto de venta (RRHH,seguimiento y gestión inventarios, bodegas, ventas, etc.). Ofrecemos atractivas comisiones

Los montos líquidos PROMEDIO mensual a los que puedes acceder, posterior a los gastos de administración van desde: $600.000 a $1.000.000

Requisitos:
-Experiencia en ventas (dependiente o independiente)
-Disponibilidad para horarios de mall
-Motivación y orientación al logro y a la venta
-Ambición y competitividad
-Manejo conceptos básicos de contabilidad y legislación laboral
-Excelente capacidad de administrar un equipo.
-Posee una sociedad limitada (si no cuenta con esto, no es impedimento, puesto que es tramite simple que se realiza de manera on line)


Buscamos personas que deseen emprender, tengan orientación a la venta y el liderazgo, para todos ellos esta es la oportunidad para iniciar un negocio propio con el respaldo de una marca líder en su rubro.
'''


f =lematizar(des)
lematizado = ''
for word in f:
    lematizado += word +' '
lematizado.replace('\n', '')

' nos encontrar en lo búsqueda de socios comercial , parir administrar por completar importante punto de venta ( corner ) de retail de calzar .  has escuchar hablar de los franquicia y consignación ? pues este empresa trabajar a través de lo consignación , este querer decir que lo empresa te hacer entregar del punto de venta , correr con todo lo gasto de infraestructura , arrendar y todo lo mercadería y renovación de stock constante , apoyar de visual y marketing , etcétera .  buscamos vendedor que se encargar de lo administración por completar del punto de venta ( rrhh , seguimiento y gestión inventario , bodega , venta , etcétera ) . ofrecemos atractivo comisionar  los monto líquido promedio mensual a lo que poder acceder , posterior a lo gasto de administración ir desde : $ 600.000 a $ 1.000.000  requisitos :  -experiencia en venta ( dependiente o independiente )  -disponibilidad parir horario de mall  -motivación y orientación al lograr y a lo venta  -ambición y competitividad  -ma

In [30]:
import os
import nltk
import string
from nltk.tokenize import sent_tokenize, word_tokenize
from spacy.matcher import Matcher
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 

def pre_process(corpus,  enminiscula= True):
    '''
    Entrada: texto, stopwords, enminiscula (opcional)
    Salida:  texto
    Funcion que se encarga de limpiar las stopwords de un texto
    el parámetro opciona enminuscula si es verdadero,
    transforma todo el texto a miniscula y elimina stopwords que esten en minuscula.
    Cuando se usa false, el texto retornado mantendra capitalizacion original y 
    además se eliminan stop words especificas tales como: Pontificia, Universidad, Vitae, VITAE
    Notar que stop_words.txt tiene stopwords en minisculas y capitalizada.
    Esta propiedad de mantener la capitalización es útil en la detección de nombres.
    '''
    newStopWords = cargar_dict('/home/erwin/Genoma/cv-parser/parser/diccionarios/stop_words')
    stopwords = nltk.corpus.stopwords.words('spanish')
    print(newStopWords)
    stop = stopwords.extend(newStopWords)
    

    if enminiscula:
        corpus = corpus.lower()
    stopset = stopwords+ list(string.punctuation)

    corpus = " ".join([i for i in word_tokenize(corpus) if i not in stopset])
    # remove non-ascii characters
    #corpus = unidecode.unidecode(corpus)
    return corpus

In [31]:
def cargar_dict(path):
    '''
    Utilidad para cargar los diccionarios
    '''
    with open(path) as f:  
        array = [x.strip() for x in f]
        c = [x for x in array if x != ''] # '' aparece cuando hay lines vacias
    return c

file = '/home/erwin/Genoma/cv-parser/parser/Outputs/output_text/1586903416737-NewHTJ'
cv_txt = cargar_dict(file)

In [32]:
t = " ".join(cv_txt)


In [33]:
pre_process(t, False)

FileNotFoundError: [Errno 2] No such file or directory: '/home/erwin/Genoma/cv-parser/parser/diccionarios/stop_words'